# Deep Learning Time Series
The hardest part is generally going to be shaping the data correctly. In doing so, you turn a time series problem into a general supervised learning problem. There are some caveats though. There has to be a constant series of data, meaning no breaks in time. Each row of the data needs to represent the same increment of time, i.e. day, hours, minutes etc.

In order to accomplish this there are some great packages in R (and I'm sure Python as well) to do some time series imputation. Spline imputation is always a good choice to imputing missing data, but if you know the seasonality of the time series you can actually leverage more advanced techniques to do some very accurate imputation. The package I use most often is imputeTS which makes it really easy to visualize and impute missing data. There are also some tricks I use to make sure that there is a constant series in time. I will put together some R notebooks with some sample code/functions but once you get the idea of what needs to be done it should be fairly easy to do this in any language.

In [5]:
from math import sqrt
from numpy import concatenate
import numpy as np
from matplotlib import pyplot
import os

import pandas as pd

from scipy.stats import uniform

from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.models import Sequential, Model
from keras.layers import Activation, Input, Dense, Conv1D, MaxPooling1D, AveragePooling1D, Dropout, Flatten, GRU, LSTM
from keras.layers.advanced_activations import ELU
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import *
from keras import backend as K
K.set_image_dim_ordering('th')

import tensorflow as tf
from pymlx import *

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# from IPython.display import Audio

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
def make_windowed(dataset, seq_length, horizon, feat_cols, resp_cols, region_col, split=0.2, resp_width=0,
                        predict_current=False):
    """
    This function takes data and creates a windowed dataframe to be used in time-series analysis. 
    dataset: [panda df] the raw dataframe that a time series windowed df will be created from
    seq_length: [int] the sequence length, the amount of time to be used to perform the 
                time-series analysis
    horizon: [int] how far into the future you wish to predict
    feat_cols: [list] a list of column names that make up the feature space
    resp_cols: [list] a list of column names that make up the response
    region_col: [str] the name of the column that different time-series will be created on, i.e. 
                different regions that contain
                independent time-series.
    split: [float] A percent split for the test set, i.e. 0.2 equals a 80/20 split for the 
                train/test sets.
    resp_width: [int] If you want to predict out to a set distance you set the horizon to that 
                    time point and this value to 0, however if you want to predict every value 
                    between let's say now and some point in the future you set horizon to 1 and
                    the resp_width to that point. The algorithm will then predict every time point.
    predict_current: [bool] horizon needs to be set to 1 for this to work. This will predict at 
                    the current time so if there is a sequence length of 2 instead of forecasting 
                    out the horizon length, the model
                    will predict at the current time.
    """

    if ((predict_current) and (horizon is not 1)):
        raise ValueError('If predict_current is set to True, then Horizon must be set to 1.')
    else:
        pass

    if (any(dataset[feat_cols + resp_cols].isnull().sum()) != 0):
        raise ValueError('There is missing data in at least one of the columns supplied in keep_cols. \
                         Please impute this missing data as needed.')
    else:
        pass

    # check to see if there are same features in both the response and the features list
    resp_and_feats = [var for var in feat_cols if var in resp_cols]

    regions = list(dataset[region_col].unique())
    big_dict = {}

    for i in range(len(regions)):
        big_dict.update({regions[i]: dataset[dataset[region_col] == regions[i]]})
    features = len(feat_cols)
    response = len(resp_cols)
    if resp_width == 0:
        train_X_all, test_X_all = np.empty((0, seq_length, features)), np.empty((0, seq_length, features))
        train_y_all, test_y_all = np.empty((0, response)), np.empty((0, response))
    else:
        train_X_all, test_X_all = np.empty((0, sequence_length, features)), np.empty((0, sequence_length, features))
        train_y_all, test_y_all = np.empty((0, resp_width, response)), np.empty((0, resp_width, response))

    for region in regions:
        big_dict[region] = big_dict[region][resp_cols + feat_cols]
        if resp_and_feats:
            big_dict[region] = big_dict[region].loc[:, ~big_dict[region].columns.duplicated()]
        else:
            pass

        length = len(big_dict[region])
        test_length = int(length * split)  # 20% test set

        df_x = big_dict[region][feat_cols]
        df_y = big_dict[region][resp_cols]
        ts_x = df_x.values
        ts_y = df_y.values

        train_length = length - test_length

        train_X, train_y, test_X, test_y = [], [], [], []

        if (predict_current):
            z = 2
            q = 1
        else:
            z = 1
            q = 0

        if (resp_width != 0):
            for i in range(train_length - sequence_length - horizon - resp_width):
                train_X.append(ts_x[i:i + sequence_length])
                train_y.append(ts_y[i + sequence_length + horizon - z:i + sequence_length + horizon - z + resp_width])
            for i in range(-test_length, -resp_width):
                test_X.append(ts_x[i - sequence_length - horizon + 1:i - horizon + 1])
                test_y.append(ts_y[i - q:i - q + (resp_width)])

        else:
            for i in range(train_length - seq_length - horizon):
                train_X.append(ts_x[i:i + seq_length])
                train_y.append(ts_y[i + seq_length + horizon - z])
            for i in range(-test_length, 0):
                test_X.append(ts_x[i - seq_length - horizon + 1:i - horizon + 1])
                test_y.append(ts_y[i - q])

        train_X, train_y = np.array(train_X), np.array(train_y)
        test_X, test_y = np.array(test_X), np.array(test_y)

        train_X_all = np.append(train_X_all, train_X, axis=0)
        test_X_all = np.append(test_X_all, test_X, axis=0)
        train_y_all = np.append(train_y_all, train_y, axis=0)
        test_y_all = np.append(test_y_all, test_y, axis=0)

    # normalize
    mean_x = train_X_all.mean(0)
    std_x = train_X_all.std(0)

    train_X = (train_X_all - mean_x) / std_x
    test_X = (test_X_all - mean_x) / std_x
    train_X = train_X.astype('float32')
    test_X = test_X.astype('float32')

    mean_y = train_y_all.mean(0)
    std_y = train_y_all.std(0)

    train_y = (train_y_all - mean_y) / std_y
    test_y = (test_y_all - mean_y) / std_y
    train_y = train_y.astype('float32')
    test_y = test_y.astype('float32')

    if resp_width != 0:
        std_y = std_y.ravel()
        mean_y = mean_y.ravel()
        train_y = train_y.reshape(train_y.shape[0], train_y.shape[1] * response)
        test_y = test_y.reshape(test_y.shape[0], test_y.shape[1] * response)
    else:
        pass

    print('train_X shape:', np.shape(train_X))
    print('train_y shape:', np.shape(train_y))
    print('test_X shape:', np.shape(test_X))
    print('test_y shape:', np.shape(test_y))

    return train_X, test_X, train_y, test_y, mean_x, std_x, mean_y, std_y

In [7]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))


def compute_mape_minMax(model, x, y):
    forecasts = model.predict(x, batch_size=len(x))
    forecasts = (forecasts *  (max_soilT - min_soilT)) + min_soilT
    y_denom = (y * (max_soilT - min_soilT)) + min_soilT
    return np.mean(np.abs((y_denom - forecasts) / y_denom))


def compute_mape_centerScale(model, x, y):
    forecasts = model.predict(x)
#     forecasts = forecasts.reshape(forecasts.shape[0],dense)
    forecasts = (forecasts * std_y) + mean_y
    y_denom = (y * std_y) + mean_y
    sub_result = np.abs((y_denom - forecasts) / y_denom )
    # remove rows that are divided by 0 or a very small number
    idxinf = np.where(sub_result == np.inf)
    idx = np.where(sub_result > 1)
    total_remove = len(idxinf[0]) + len(idx[0])
    #print('Removed %f percent of the data, a total of %d rows.'% (((total_remove/len(forecasts))*100), total_remove))
    sub_result = np.delete(sub_result, idxinf[0],0)
    sub_result = np.delete(sub_result, idx[0], 0)
    result = (np.mean(sub_result))
    return result


def compute_mae(model, x, y):
    forecasts = model.predict(x)
#     forecasts = forecasts.reshape(forecasts.shape[0],dense)
    forecasts = (forecasts * std_y) + mean_y
    y_denom = (y * std_y) + mean_y
    result = mean_absolute_error(y_pred=forecasts, y_true=y_denom)
    return result


def test(model):
    # print('Train Mean Absolute Percentage Error (MAPE): {0:.3f}'.format(compute_mape_centerScale(model, train_X, train_y)))
    # print('Test Mean Absolute Percentage Error (MAPE): {0:.3f}'.format(compute_mape_centerScale(model, test_X, test_y)))
    return(compute_mape_centerScale(model, train_X, train_y), compute_mape_centerScale(model, test_X, test_y))

def test_mae(model):
    yhat_train = model.predict(train_X)
    yhat_test = model.predict(test_X)
    return(compute_mae(model, train_X, train_y), compute_mae(model, test_X, test_y))


def report(train, test):
    train_mean = np.asarray(train).mean()
    train_std = np.asarray(train).std()
    test_mean = np.asarray(test).mean()
    test_std = np.asarray(test).std()
    train = str('Training MAE %f ± %f'% (train_mean, train_std))
    test = str('Testing MAE %f ± %f'% (test_mean, test_std))
    return(train, test)

In [8]:
CWD = os.getcwd()
# FILE_PATH = os.path.join(CWD, "../data/time_series_soilM_averaged_50percent_tuning.csv")
FILE_PATH = os.path.join(CWD, "../data/time_series_soilM_averaged.csv")
dataset = pd.read_csv(FILE_PATH)
dataset.head()
dataset['date'] = pd.to_datetime(dataset['date'])
dataset = dataset.set_index(['date'])
dataset = dataset.reset_index()

,date,field,soilM_avg,imputed,precip.cm,tair.C,rh.pct,wind_sp.m_per_s,irradiance.w_per_m.2,average_sand,average_silt,average_clay
0,2011-10-29,AES,0.212173,0.212173,0.045,7.654167,68.083333,3.179167,121.708333,14.891866,60.796955,24.311179
1,2011-10-30,AES,0.210993,0.210993,0.369,9.354167,78.250000,3.562500,72.500000,14.891866,60.796955,24.311179
2,2011-10-31,AES,0.209983,0.209983,0.023,6.945833,68.625000,5.112500,122.666667,14.891866,60.796955,24.311179
3,2011-11-01,AES,0.209532,0.209532,0.000,2.225000,66.250000,3.220833,127.500000,14.891866,60.796955,24.311179
4,2011-11-02,AES,0.207929,0.207929,0.000,4.862500,44.958333,6.091667,124.041667,14.891866,60.796955,24.311179


### Set Parameters

In [9]:
resp_cols = ['imputed']
feat_cols = ['precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 'irradiance.w_per_m.2', 'average_clay',
       'average_sand', 'average_silt']
keep_cols = resp_cols + feat_cols

sequence_length = 30
horizon = 14
resp_width = 0
dense = len(resp_cols)

### Make Windowed DF

In [10]:
train_X, test_X, train_y, test_y, mean_x, std_x, mean_y, std_y = make_windowed(dataset=dataset, 
                                                                                     seq_length=sequence_length,
                                                                                     horizon=horizon,
                                                                                     resp_cols=resp_cols,
                                                                                     feat_cols=feat_cols,
                                                                                     region_col='field',
                                                                                     resp_width=resp_width,
                                                                                     predict_current=False)
dim = train_X.shape[2]

train_X shape: (5244, 30, 8)
train_y shape: (5244, 1)
test_X shape: (1364, 30, 8)
test_y shape: (1364, 1)


# CNN
## Create a function that takes as input all the parameters for the deep learning model.

In [12]:
np.random.seed(84)
tf.set_random_seed(84)

def create_tCNN_model(max_meanPool_1=1,
                      max_meanPool_2=1,
                      max_meanPool_3=1,
                      max_meanPool_4=1,
                      max_meanPool_5=1,
                      max_meanPool_6=1,
                      filters=32, 
                      kernel_size=1, 
                      pool_size=1, 
                      stride=1, 
                      optimizer='RMSprop', 
                      layers=1,
                      init='glorot_normal',
                      lr=0.001, 
                      momentum=0.9,
                      decay=0.9,
                      beta_1=0.9,
                      beta_2=0.999,
                      rho=0.9,
                      nesterov=False,
                      activation = ELU(),
                      dropout_rate = 0.0,
                      epsilon=1e-08,
                      schedule_decay=0.004):
    
    sequence_input = Input(shape=(sequence_length, dim))
            
    if(layers == 1):
        x = Conv1D(filters, kernel_size)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
    elif(layers == 2):
        x = Conv1D(filters, kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_2==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_2==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
    elif(layers == 3):
        x = Conv1D(filters, kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_2==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_2==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_3==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_3==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
    elif(layers == 4):
        x = Conv1D(filters, kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_2==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_2==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_3==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_3==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2/2/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_4==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_4==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
    elif(layers == 5):
        x = Conv1D(filters, kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_2==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_2==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_3==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_3==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2/2/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_4==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_4==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2/2/2/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_5==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_5==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
    else:
        x = Conv1D(filters, kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_2==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_2==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_3==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_3==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2/2/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_4==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_4==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2/2/2/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_5==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_5==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
        x = Conv1D(int(filters/2/2/2/2/2), kernel_size, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_6==1):
            x = MaxPooling1D(pool_size=pool_size, stride=stride)(x)
        elif(max_meanPool_6==2):  
            x = AveragePooling1D(pool_size=pool_size, stride=stride)(x)
        else:
            pass
        
    x = Flatten()(x)
    x = Dense(filters, init=init)(x)
    x = activation(x)
    x = Dropout(dropout_rate)(x)
    preds = Dense(dense)(x)
    model = Model(sequence_input, preds)

    if(optimizer == 'SGD'):
        sgd = SGD(lr=lr, momentum=momentum, decay=decay, nesterov=nesterov)
    elif(optimizer == 'RMSprop'):
        sgd = RMSprop(lr=lr, rho=rho)
    elif(optimizer == 'Nadam'):
        sgd = Nadam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, schedule_decay=schedule_decay)
    elif(optimizer == 'Adagrad'):
        sgd = Adagrad(lr=lr, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adadelta'):
        sgd = Adadelta(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adam'):
        sgd = Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adamax'):
        sgd = Adamax(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay)
    else:
        pass
    
    model.compile(loss='mae', optimizer=sgd, metrics=['mse'])
    
    return model

## Build the parameter tuning space that you will tune over

In [15]:
print('Forecasting out {0} days, with a {1:0.1f} day window. '.format(horizon, (sequence_length)))
print('Feature space: ', feat_cols)
print('Response: ', resp_cols)

model = KerasRegressor(build_fn=create_tCNN_model, shuffle=True, verbose=1)

# init = ['uniform', 'normal', 'zero', 'glorot_uniform', 'glorot_normal', 'lecun_uniform', 
#                       'lecun_normal', 'he_normal', 'he_uniform']
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

layers = [1,2,3,4,5,6]

nesterov = [False]
filters = [32,64,128,256,512] # 32 is the lowest number of filters for tuning up to 6 layers
batch_size = [2,4,8,16,32,64]
epochs = [5]
lr = [10**k for k in range(-5, -1)]
max_meanPool_1=[1,2]
max_meanPool_2=[1,2]
max_meanPool_3=[1,2]
max_meanPool_4=[1,2]
max_meanPool_5=[1,2]
max_meanPool_6=[1,2]
kernel_size=[1,2,3,4]
pool_size=[1,2,3,4]
stride=[1,2,3,4]

optimizer = ['Adam', 'Nadam', 'Adamax', 'RMSprop']
init = ['glorot_uniform', 'glorot_normal', 'lecun_uniform', 'lecun_normal']
momentum=[0.9]
dropout_rate=[0.0]
decay = np.arange(0,.1,0.001).tolist()
# decay = np.arange(0.06,0.1,0.001).tolist()
beta_1 = np.random.uniform(low=0.90, high=1.0, size=20).tolist()
beta_2 = np.random.uniform(0.90, 1.0, 20).tolist()
rho = np.random.uniform(0.80, 0.95, 20).tolist()
epsilon = [1e-09, 1e-08,1e-07, 1e-06, 1e-05, 1e-04]
schedule_decay = np.arange(0.001,0.01,0.0005).tolist()

param_grid = dict(batch_size=batch_size, 
                  epsilon=epsilon,
                  schedule_decay=schedule_decay,
                  epochs=epochs, 
                  layers=layers, 
                  filters=filters,
                  optimizer=optimizer,
                  lr=lr,
                  decay=decay,
                  init=init,
                  beta_1=beta_1,
                  beta_2=beta_2,
                  nesterov=nesterov,
                  rho=rho,
                  pool_size=pool_size,
                  kernel_size=kernel_size,
                  stride=stride,
                  momentum=momentum,
                  dropout_rate=dropout_rate,
                  max_meanPool_1=max_meanPool_1, 
                  max_meanPool_2=max_meanPool_2, 
                  max_meanPool_3=max_meanPool_3, 
                  max_meanPool_4=max_meanPool_4, 
                  max_meanPool_5=max_meanPool_5,
                  max_meanPool_6=max_meanPool_6)

start_timing()
sweeper = random_sweep(
    train_X, train_y, 
    model, param_grid,
    scoring=compute_mae, 
    n_iter=1, n_jobs=1, 
    refit=False, cv=3, verbose=1)
report_timing()


Forecasting out 14 days, with a 30.0 day window. 
Feature space:  ['precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 'irradiance.w_per_m.2', 'average_clay', 'average_sand', 'average_silt']
Response:  ['imputed']
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Epoch 1/5
3496/3496 [==============================] - 1s 167us/step - loss: 0.6306 - mean_squared_error: 0.7320
Epoch 2/5
3496/3496 [==============================] - 0s 45us/step - loss: 0.5370 - mean_squared_error: 0.5742
Epoch 3/5
3496/3496 [==============================] - 0s 41us/step - loss: 0.5130 - mean_squared_error: 0.5316
Epoch 4/5
3496/3496 [==============================] - 0s 42us/step - loss: 0.5001 - mean_squared_error: 0.5110
Epoch 5/5
3496/3496 [==============================] - 0s 21us/step
Epoch 1/5
3496/3496 [==============================] - 1s 162us/step - loss: 0.6482 - mean_squared_error: 0.7460
Epoch 2/5
3496/3496 [==============================] - 0s 44us/step - loss: 0.5630 - mean_squared

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.9s finished


In [ ]:
results = sweep_stats(sweeper)
FILE_PATH = os.path.join(CWD, "../results/best_params/forecasting/tCNN_hourlyData-25percent_SEEDFEATS-DP-RH-WindGust-Speed_Resp-ForeCast-LW-8-2-VWC_H1_Rw120_W360_Tune_run1.csv")
results.to_csv(FILE_PATH)
results

# LSTM/GRU
## Same approach as above just with a different model. If you want to run a GRU, just change LSTM to GRU on line 23. GRU's generally run faster than LSTM but essentially the same backend. 

In [ ]:
print('Forecasting out {0} days, with a {1} day window. '.format(horizon, sequence_length))
print('Feature space: ', feat_cols)
print('Response: ', resp_cols)
np.random.seed(84)
tf.set_random_seed(84)

def create_LSTM_model(layers=1, 
                 optimizer='adam', 
                 units=5, 
                 lr=0.001, 
                 momentum=0.9, 
                 decay=0.9,
                 kernel_initializer='glorot_normal',
                 beta_1=0.9,
                 beta_2=0.999,
                 rho=0.9):
    
    np.random.seed(84)
    tf.set_random_seed(84)
    
    model = Sequential()
    for i in range(layers):
        model.add(LSTM(units, input_shape=(sequence_length, dim_x),
                       return_sequences=True if i < layers-1 else False))
        
    model.add(Dense(dim_y, kernel_initializer=kernel_initializer))
    
    if(optimizer == 'SGD'):
        sgd = SGD(lr=lr, momentum=momentum, decay=decay, nesterov=False)
    elif(optimizer == 'RMSprop'):
        sgd = RMSprop(lr=lr, rho=rho)
    elif(optimizer == 'Nadam'):
        sgd = Nadam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=1e-08, schedule_decay=0.004)
    elif(optimizer == 'Adagrad'):
        sgd = Adagrad(lr=lr, epsilon=1e-08, decay=decay)
    elif(optimizer == 'Adadelta'):
        sgd = Adadelta(lr=lr, rho=0.95, epsilon=1e-08, decay=decay)
    elif(optimizer == 'Adam'):
        sgd = Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=1e-08, decay=decay)
    elif(optimizer == 'Adamax'):
        sgd = Adamax(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=1e-08, decay=decay)
    else:
        pass
    
    model.compile(loss='mae', optimizer=sgd, metrics=['mse'])
    
    return model

In [ ]:
model = KerasRegressor(build_fn=create_model, shuffle=True, verbose=0)

# kernel_initializer = ['uniform', 'normal', 'zero', 'glorot_uniform', 'glorot_normal', 'lecun_uniform', 
#                       'lecun_normal', 'he_normal', 'he_uniform']
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

layers = [1]
units = [2,4,8,16,32,64,128,256,512]
batch_size = [2,4,8,16,32]
epochs = [50]
# lr = [0.0005, 0.001, 0.005, 0.01, 0.05]
lr = [10**k for k in range(-5, -1)]
beta_1 = np.random.uniform(low=0.9, high=1.0, size=20).tolist()
# beta_1 = [0.9]
beta_2 = np.random.uniform(0.95, 1.0, 20).tolist()
# beta_2 = [0.999]
decay = np.arange(0,.2,0.01).tolist()
rho = np.random.uniform(0.8, 1.0, 20).tolist()
# decay = [0, 0.05, 0.01, 0.015, 0.02, 0.025]
# decay = np.concatenate([np.arange(0,.1,0.01).tolist(), np.arange(0.95,1,0.01).tolist()]).tolist()

kernel_initializer = ['normal', 'zero', 'glorot_uniform', 'glorot_normal', 'lecun_uniform', 
                       'lecun_normal', 'he_normal', 'he_uniform']
optimizer = ['RMSprop', 'Adam', 'Adamax', 'Nadam']

param_grid = dict(batch_size=batch_size, 
                  epochs=epochs, 
                  layers=layers, 
                  units=units,
                  optimizer=optimizer,
                  lr=lr,
                  decay=decay,
                  kernel_initializer=kernel_initializer,
                  beta_1=beta_1,
                  beta_2=beta_2)

start_timing()
sweeper = random_sweep(
    train_X, train_y, 
    model, param_grid,
    scoring=compute_mae, 
    n_iter=15, n_jobs=6, refit=False, cv=tscv, verbose=1)
report_timing()